In [28]:
!pip install pybooru
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 2.3 MB/s eta 0:00:00a 0:00:01


In [31]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta
import time

# Setup headless Firefox
options = Options()
options.add_argument('--headless')

# You can specify executable_path if not in PATH:
driver = webdriver.Firefox(options=options)  # or Firefox(executable_path="path/to/geckodriver", options=options)

# Load weekly bounds (you can remove .head(1) later)
df = pd.read_csv("weekly_bounds.csv")
estimated_counts = []

for _, row in df.iterrows():
    start_date = datetime.strptime(row['reporting_week'], "%Y-%m-%d")
    end_date = start_date + timedelta(days=6)
    total_weekly_posts = 0

    for i in range(7):
        day = (start_date + timedelta(days=i)).strftime('%Y-%m-%d')
        url = f"https://danbooru.donmai.us/posts?tags=date:{day}"
        print(f"Checking {day}...")
        driver.get(url)
        time.sleep(2)  # Wait for JavaScript/rendering

        # Default to 1 page unless more are found
        pages = 1

        try:
            # Find all numbered paginator links
            page_elements = driver.find_elements(By.CSS_SELECTOR, "a.paginator-page")
            page_numbers = [int(a.text.strip()) for a in page_elements if a.text.strip().isdigit()]
            if page_numbers:
                pages = max(page_numbers)
        except Exception as e:
            print(f"Error parsing {day}: {e}")

        print(f"  → {pages} pages")
        total_weekly_posts += pages * 20

    estimated_counts.append(total_weekly_posts)

driver.quit()

# Save to CSV
df["estimated_posts"] = estimated_counts
df.to_csv("weekly_estimated_post_counts.csv", index=False)


Checking 2009-01-04...
  → 35 pages
Checking 2009-01-05...
  → 31 pages
Checking 2009-01-06...
  → 32 pages
Checking 2009-01-07...
  → 30 pages
Checking 2009-01-08...
  → 30 pages
Checking 2009-01-09...
  → 27 pages
Checking 2009-01-10...
  → 28 pages
Checking 2009-01-11...
  → 30 pages
Checking 2009-01-12...
  → 31 pages
Checking 2009-01-13...
  → 28 pages
Checking 2009-01-14...
  → 28 pages
Checking 2009-01-15...
  → 29 pages
Checking 2009-01-16...
  → 28 pages
Checking 2009-01-17...
  → 29 pages
Checking 2009-01-18...
  → 30 pages
Checking 2009-01-19...
  → 37 pages
Checking 2009-01-20...
  → 24 pages
Checking 2009-01-21...
  → 26 pages
Checking 2009-01-22...
  → 19 pages
Checking 2009-01-23...
  → 32 pages
Checking 2009-01-24...
  → 28 pages
Checking 2009-01-25...
  → 31 pages
Checking 2009-01-26...
  → 25 pages
Checking 2009-01-27...
  → 24 pages
Checking 2009-01-28...
  → 26 pages
Checking 2009-01-29...
  → 20 pages
Checking 2009-01-30...
  → 27 pages
Checking 2009-01-31...
  → 2

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=57843): Read timed out. (read timeout=120)

In [27]:
print(f"Status code for {day}: {response.status_code}")
print(response.url)
print(response.text[:500])  # Show a snippet of the HTML


Status code for 2009-01-10: 403
https://danbooru.donmai.us/posts?tags=date%3A2009-01-10
<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetic
